In [ ]:
# 生成对抗是一种非监督学习，通过生成器和判别器进行对抗博弈学习过程，最终提高双方生成能力和判别能力
# 源于Lan Goodfellow 在2014年提出了GAN

# 博弈论中极大极小思想： max_g min_d d(g(z))
# 交替训练

# 数据归一化，保证训练数据分布稳定
# torch.nn.BatchNorm1d, torch.nn.BatchNorm2d, torch.nn.BatchNorm3d
# torch.nn.InstanceNorm1d, torch.nn.InstanceNorm2d, torch.nn.InstanceNorm3d
# torch.nn.LayerNorm

# 初始化
# torch.nn.init.uniform_(), torch.nn.init.normal_(), torch.nn.init.constant_()
# torch.nn.init.xavier_uniform_(), torch.nn.init.xavier_normal_()


In [2]:
# 数据

from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.utils import save_image

dataset = CIFAR10(root='./data', download=True, transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [13]:
import torch
# 生成器
gnet = torch.nn.Sequential(
    # (64,1,1)
    torch.nn.ConvTranspose2d(64, 4 * 64, kernel_size=4, bias=False),
    torch.nn.BatchNorm2d(4 * 64),
    torch.nn.ReLU(),
    # (256, 4, 4)
    torch.nn.ConvTranspose2d(4 * 64, 2*64, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(2*64),
    torch.nn.ReLU(),
    # (128,8,8)
    torch.nn.ConvTranspose2d(2*64, 64, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),
    # (64,16,16)
    torch.nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2,padding=1),
    torch.nn.Sigmoid()
    # (3, 32, 32)
    )

print('gnet', gnet)

# 判别器
dnet = torch.nn.Sequential(
    # (3,32,32)
    torch.nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
    torch.nn.LeakyReLU(0.2),
    # (64, 16, 16)
    torch.nn.Conv2d(64, 2*64, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(2*64),
    torch.nn.LeakyReLU(0.2),
    # (128, 8, 8)
    torch.nn.Conv2d(2*64, 4*64, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(4*64),
    torch.nn.LeakyReLU(0.2),
    # (256,4,4)
    torch.nn.Conv2d(4*64, 1, kernel_size=4)
    )

print('dnet:', dnet)


# 采用特殊初始化网络实例

def weight_init(para):
    if type(para) in [torch.nn.ConvTranspose2d, torch.nn.Conv2d]:
        torch.nn.init.xavier_normal_(para.weight)
    elif type(para) == torch.nn.BatchNorm2d:
        torch.nn.init.normal_(para.weight, 1.0, 0.02)
        torch.nn.init.constant_(para.bias, 0)

gnet.apply(weight_init)
dnet.apply(weight_init)

gnet Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): Sigmoid()
)
dnet: Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4):

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)

In [17]:
# 训练

criterion = torch.nn.BCEWithLogitsLoss()
goptimizer = torch.optim.Adam(gnet.parameters(), lr=0.0002, betas=(0.5,0.999))
doptimizer = torch.optim.Adam(dnet.parameters(), lr=0.0002, betas=(0.5,0.999))


for epoch in range(10):
    for batch_idx, data in enumerate(dataloader):
        # 无监督训练
        real_images, _ = data
        batch_size = real_images.size(0)
        
        # 真实数据
        real_labels = torch.ones(batch_size)
        real_preds = dnet(real_images).reshape(-1)
        real_dloss = criterion(real_preds, real_labels)
        real_dmean = real_preds.sigmoid().mean()
        
        # 生成数据
        noise_input = torch.randn(batch_size, 64, 1, 1)
        fake_images = gnet(noise_input)
        fake_labels_ = torch.zeros(batch_size)
        fake_images_ = fake_images.detach()
        fake_preds_ = dnet(fake_images_).view(-1)
        fake_dloss_ = criterion(fake_preds_, fake_labels_)
        fake_dmean = fake_preds_.sigmoid().mean()
        
        # 训练判别器：同时优化真实数据和生成数据
        dloss = real_dloss + fake_dloss_
        doptimizer.zero_grad()
        dloss.backward()
        doptimizer.step()
        
        # 训练生成器：优化生成数据
        fake_labels = torch.ones(batch_size)
        fake_preds = dnet(fake_images).view(-1)
        gloss = criterion(fake_preds, fake_labels)
        goptimizer.zero_grad()
        gloss.backward()
        goptimizer.step()
        
        if batch_idx%10==0:
            print('第{}迭代下第{}批次: 判别损失={}，生成损失={}, real_dmean={}, fake_dmean={}'.
              format(epoch, batch_idx, dloss, gloss, real_dmean, fake_dmean))
            

第0迭代下第0批次: 判别损失=0.08434556424617767，生成损失=6.563366889953613, real_dmean=0.9645393490791321, fake_dmean=0.04231749102473259
第0迭代下第10批次: 判别损失=0.22085706889629364，生成损失=3.5826005935668945, real_dmean=0.8684533834457397, fake_dmean=0.06666837632656097
第0迭代下第20批次: 判别损失=0.10985997319221497，生成损失=4.784465312957764, real_dmean=0.9070792198181152, fake_dmean=0.0060376557521522045
第0迭代下第30批次: 判别损失=0.23157250881195068，生成损失=3.708918809890747, real_dmean=0.8714855313301086, fake_dmean=0.07131540775299072
第0迭代下第40批次: 判别损失=0.2764389216899872，生成损失=5.2186994552612305, real_dmean=0.8834843635559082, fake_dmean=0.12246251851320267
第0迭代下第50批次: 判别损失=0.35679957270622253，生成损失=3.856228828430176, real_dmean=0.7877732515335083, fake_dmean=0.07124626636505127
第0迭代下第60批次: 判别损失=0.6028554439544678，生成损失=4.091498851776123, real_dmean=0.8671908378601074, fake_dmean=0.30802032351493835
第0迭代下第70批次: 判别损失=0.628990888595581，生成损失=3.0334861278533936, real_dmean=0.7279706597328186, fake_dmean=0.1671130657196045
第0迭代下第80批次: 判别损失=

第0迭代下第670批次: 判别损失=0.9728689193725586，生成损失=2.024376392364502, real_dmean=0.6656652092933655, fake_dmean=0.36594343185424805
第0迭代下第680批次: 判别损失=0.8210138082504272，生成损失=2.2558977603912354, real_dmean=0.8307033181190491, fake_dmean=0.4422130584716797
第0迭代下第690批次: 判别损失=1.0184245109558105，生成损失=2.7046375274658203, real_dmean=0.7788686156272888, fake_dmean=0.488380491733551
第0迭代下第700批次: 判别损失=0.7535871267318726，生成损失=2.042137622833252, real_dmean=0.6455379128456116, fake_dmean=0.22206932306289673
第0迭代下第710批次: 判别损失=0.9356599450111389，生成损失=1.5789915323257446, real_dmean=0.6154754161834717, fake_dmean=0.2887496054172516
第0迭代下第720批次: 判别损失=0.9302873611450195，生成损失=1.9376522302627563, real_dmean=0.6238278150558472, fake_dmean=0.3105897307395935
第0迭代下第730批次: 判别损失=1.1668555736541748，生成损失=1.2689638137817383, real_dmean=0.4140392541885376, fake_dmean=0.1266816407442093
第0迭代下第740批次: 判别损失=1.0153424739837646，生成损失=2.475417375564575, real_dmean=0.8201665282249451, fake_dmean=0.5270996689796448
第0迭代下第750批次: 判别损失=

第1迭代下第550批次: 判别损失=0.7880003452301025，生成损失=2.1174356937408447, real_dmean=0.7219080924987793, fake_dmean=0.3406107425689697
第1迭代下第560批次: 判别损失=0.832095205783844，生成损失=1.8132213354110718, real_dmean=0.6897485256195068, fake_dmean=0.3335244953632355
第1迭代下第570批次: 判别损失=0.607383131980896，生成损失=1.9865448474884033, real_dmean=0.8406447172164917, fake_dmean=0.3265215754508972
第1迭代下第580批次: 判别损失=0.6797335147857666，生成损失=2.1243655681610107, real_dmean=0.7751823663711548, fake_dmean=0.32263368368148804
第1迭代下第590批次: 判别损失=1.0285497903823853，生成损失=1.0167251825332642, real_dmean=0.5288244485855103, fake_dmean=0.2532396614551544
第1迭代下第600批次: 判别损失=0.7317360639572144，生成损失=1.726452112197876, real_dmean=0.614475667476654, fake_dmean=0.18280287086963654
第1迭代下第610批次: 判别损失=0.765508770942688，生成损失=1.9510304927825928, real_dmean=0.7270298004150391, fake_dmean=0.3237536549568176
第1迭代下第620批次: 判别损失=0.8204450607299805，生成损失=1.9755760431289673, real_dmean=0.7674299478530884, fake_dmean=0.39978668093681335
第1迭代下第630批次: 判别损失=

第2迭代下第430批次: 判别损失=0.9122008681297302，生成损失=1.560577154159546, real_dmean=0.6328079700469971, fake_dmean=0.3111151158809662
第2迭代下第440批次: 判别损失=0.9204719662666321，生成损失=1.9627485275268555, real_dmean=0.6499660015106201, fake_dmean=0.34376439452171326
第2迭代下第450批次: 判别损失=1.017236590385437，生成损失=2.671506881713867, real_dmean=0.8327963352203369, fake_dmean=0.5308167338371277
第2迭代下第460批次: 判别损失=0.7198184728622437，生成损失=1.8169605731964111, real_dmean=0.6712724566459656, fake_dmean=0.23811718821525574
第2迭代下第470批次: 判别损失=0.7290027737617493，生成损失=2.144355058670044, real_dmean=0.8034422397613525, fake_dmean=0.37245216965675354
第2迭代下第480批次: 判别损失=0.812409520149231，生成损失=1.2901265621185303, real_dmean=0.5681913495063782, fake_dmean=0.1707356870174408
第2迭代下第490批次: 判别损失=0.7098806500434875，生成损失=1.904081106185913, real_dmean=0.714042067527771, fake_dmean=0.27020251750946045
第2迭代下第500批次: 判别损失=0.8998031616210938，生成损失=1.4970331192016602, real_dmean=0.6434884071350098, fake_dmean=0.3181506097316742
第2迭代下第510批次: 判别损失=0

第3迭代下第310批次: 判别损失=0.9825531244277954，生成损失=1.5313071012496948, real_dmean=0.5475177764892578, fake_dmean=0.24575954675674438
第3迭代下第320批次: 判别损失=0.8213165998458862，生成损失=1.0514438152313232, real_dmean=0.5614204406738281, fake_dmean=0.17029064893722534
第3迭代下第330批次: 判别损失=0.9646046161651611，生成损失=1.873784065246582, real_dmean=0.6733965277671814, fake_dmean=0.3873012363910675
第3迭代下第340批次: 判别损失=1.0451691150665283，生成损失=1.1312097311019897, real_dmean=0.5070217847824097, fake_dmean=0.2261974811553955
第3迭代下第350批次: 判别损失=1.158441185951233，生成损失=0.8278856873512268, real_dmean=0.3864557445049286, fake_dmean=0.10617391020059586
第3迭代下第360批次: 判别损失=0.9144200086593628，生成损失=1.881060242652893, real_dmean=0.6872055530548096, fake_dmean=0.36352360248565674
第3迭代下第370批次: 判别损失=0.904952883720398，生成损失=1.8632904291152954, real_dmean=0.6952926516532898, fake_dmean=0.38598236441612244
第3迭代下第380批次: 判别损失=0.9134103059768677，生成损失=1.4466370344161987, real_dmean=0.5784772634506226, fake_dmean=0.2633182108402252
第3迭代下第390批次: 判别

...

第5迭代下第740批次: 判别损失=0.9955960512161255，生成损失=2.6610565185546875, real_dmean=0.8122183084487915, fake_dmean=0.4928717315196991
第5迭代下第750批次: 判别损失=0.7245989441871643，生成损失=1.7725002765655518, real_dmean=0.7384385466575623, fake_dmean=0.3000933527946472
第5迭代下第760批次: 判别损失=0.9296326637268066，生成损失=1.8041081428527832, real_dmean=0.7109522819519043, fake_dmean=0.4042365849018097
第5迭代下第770批次: 判别损失=0.6887548565864563，生成损失=1.6662681102752686, real_dmean=0.6960586309432983, fake_dmean=0.24181602895259857
第5迭代下第780批次: 判别损失=0.7468101978302002，生成损失=1.8018503189086914, real_dmean=0.7360919117927551, fake_dmean=0.3131471872329712
第6迭代下第0批次: 判别损失=0.8481423854827881，生成损失=1.3526113033294678, real_dmean=0.608686625957489, fake_dmean=0.24914707243442535
第6迭代下第10批次: 判别损失=0.6999359726905823，生成损失=2.5864973068237305, real_dmean=0.830008327960968, fake_dmean=0.3724403977394104
第6迭代下第20批次: 判别损失=0.7390673756599426，生成损失=2.0376269817352295, real_dmean=0.7663382291793823, fake_dmean=0.32783281803131104
第6迭代下第30批次: 判别损失=0.

第6迭代下第620批次: 判别损失=1.4768041372299194，生成损失=3.2617077827453613, real_dmean=0.8743359446525574, fake_dmean=0.6763903498649597
第6迭代下第630批次: 判别损失=0.9987660646438599，生成损失=2.3940279483795166, real_dmean=0.7356470227241516, fake_dmean=0.4378250241279602
第6迭代下第640批次: 判别损失=0.71767258644104，生成损失=1.6390541791915894, real_dmean=0.6536399722099304, fake_dmean=0.19398638606071472
第6迭代下第650批次: 判别损失=0.8647552728652954，生成损失=2.0424728393554688, real_dmean=0.6800578236579895, fake_dmean=0.33025333285331726
第6迭代下第660批次: 判别损失=1.0030118227005005，生成损失=2.958522319793701, real_dmean=0.8556140661239624, fake_dmean=0.5190215110778809
第6迭代下第670批次: 判别损失=0.6475604772567749，生成损失=1.9910986423492432, real_dmean=0.7440415024757385, fake_dmean=0.260722279548645
第6迭代下第680批次: 判别损失=1.0505645275115967，生成损失=0.9159897565841675, real_dmean=0.4614335894584656, fake_dmean=0.16588355600833893
第6迭代下第690批次: 判别损失=0.7386100888252258，生成损失=1.7084934711456299, real_dmean=0.6332888603210449, fake_dmean=0.18348267674446106
第6迭代下第700批次: 判别损

第7迭代下第500批次: 判别损失=0.7817825078964233，生成损失=2.8433761596679688, real_dmean=0.8380255103111267, fake_dmean=0.41652852296829224
第7迭代下第510批次: 判别损失=0.9722101092338562，生成损失=1.0388075113296509, real_dmean=0.47801464796066284, fake_dmean=0.10886398702859879
第7迭代下第520批次: 判别损失=0.7662486433982849，生成损失=1.6652510166168213, real_dmean=0.762107253074646, fake_dmean=0.3544312119483948
第7迭代下第530批次: 判别损失=1.0571739673614502，生成损失=1.440993070602417, real_dmean=0.5721375942230225, fake_dmean=0.32436734437942505
第7迭代下第540批次: 判别损失=0.6562495827674866，生成损失=2.3398561477661133, real_dmean=0.8207430839538574, fake_dmean=0.32841747999191284
第7迭代下第550批次: 判别损失=0.6464498043060303，生成损失=2.286519765853882, real_dmean=0.6749511957168579, fake_dmean=0.17576389014720917
第7迭代下第560批次: 判别损失=1.107511043548584，生成损失=0.8494018316268921, real_dmean=0.45080992579460144, fake_dmean=0.18012800812721252
第7迭代下第570批次: 判别损失=0.621394157409668，生成损失=2.0253419876098633, real_dmean=0.6930158138275146, fake_dmean=0.17843103408813477
第7迭代下第580批次:

第8迭代下第380批次: 判别损失=0.8518284559249878，生成损失=2.1590003967285156, real_dmean=0.8497174978256226, fake_dmean=0.44357213377952576
第8迭代下第390批次: 判别损失=0.8110255002975464，生成损失=1.422577977180481, real_dmean=0.571183443069458, fake_dmean=0.14598867297172546
第8迭代下第400批次: 判别损失=1.3782764673233032，生成损失=1.1884292364120483, real_dmean=0.3302604556083679, fake_dmean=0.09796693176031113
第8迭代下第410批次: 判别损失=0.8884243369102478，生成损失=1.0407238006591797, real_dmean=0.5958401560783386, fake_dmean=0.23801198601722717
第8迭代下第420批次: 判别损失=1.0260205268859863，生成损失=2.5945188999176025, real_dmean=0.7106691598892212, fake_dmean=0.42864322662353516
第8迭代下第430批次: 判别损失=0.624243974685669，生成损失=1.5938911437988281, real_dmean=0.7080392241477966, fake_dmean=0.20443816483020782
第8迭代下第440批次: 判别损失=0.7538996338844299，生成损失=1.6433192491531372, real_dmean=0.6869240999221802, fake_dmean=0.2510300874710083
第8迭代下第450批次: 判别损失=0.773456871509552，生成损失=1.9498331546783447, real_dmean=0.7210897207260132, fake_dmean=0.31284499168395996
第8迭代下第460批次: 

第9迭代下第260批次: 判别损失=0.7306243181228638，生成损失=1.661392092704773, real_dmean=0.6388769149780273, fake_dmean=0.20002338290214539
第9迭代下第270批次: 判别损失=0.7049425840377808，生成损失=2.03511905670166, real_dmean=0.7529082298278809, fake_dmean=0.3069228231906891
第9迭代下第280批次: 判别损失=1.1079535484313965，生成损失=1.1558510065078735, real_dmean=0.4830162525177002, fake_dmean=0.20679128170013428
第9迭代下第290批次: 判别损失=0.905644416809082，生成损失=1.4211890697479248, real_dmean=0.5158320665359497, fake_dmean=0.15067633986473083
第9迭代下第300批次: 判别损失=0.7936159372329712，生成损失=1.4075424671173096, real_dmean=0.6391701698303223, fake_dmean=0.23706990480422974
第9迭代下第310批次: 判别损失=0.7821336388587952，生成损失=1.7640748023986816, real_dmean=0.645266056060791, fake_dmean=0.23207689821720123
第9迭代下第320批次: 判别损失=0.9953293800354004，生成损失=2.560338020324707, real_dmean=0.8643552660942078, fake_dmean=0.5323917865753174
第9迭代下第330批次: 判别损失=0.6586422324180603，生成损失=1.9843354225158691, real_dmean=0.7260493636131287, fake_dmean=0.2507269084453583
第9迭代下第340批次: 判别损失

In [24]:
import matplotlib.pyplot as plt

# 生成新样本，看看
noise_input = torch.randn(batch_size, 64, 1, 1)
fake_images = gnet(noise_input)
fake_labels_ = torch.zeros(batch_size)
fake_images_ = fake_images.detach()
for i, fake_image_ in enumerate(fake_images_):
    # print(fake_image_.shape)
    save_image(fake_image_, './data/fake_images/{}.png'.format(i))
